In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Сингулярное разложение

В этом ДЗ есть четыре варианта заданий, с разной разбалловкой. Вы можете сделать одно из них и получить соответствующее количество баллов (из 10):  

- 4 - пакетная функция SVD с обычной матрицей
- 7 - собственное SVD с обычной матрицей
- 8 - SVD с реальным датасетом, пакетная реализация
- 10 - реальный датасет, собственная реализация

## (4-7) Типовая матрица

Постройте сингулярное разложение для матрицы A:

$$A=\left(
\begin{array}{rrr}
1 & -1 & -2 \\
-7/3 &  1/3 & 2/3\\
1/3 & -7/3 & -2/3 \\
-5/3 & 5/3 & -2/3
\end{array}
\right) \, $$

Составьте матрицы $U$, $\Sigma$, $V$ (в неусеченном виде)

При написании собственной реализации можно использовать *np.linalg.solve* и *np.linalg.inv*

In [ ]:
A = np.array([[1, -1, -2],
              [-7/3, 1/3, 2/3],
              [1/3, -7/3, -2/3],
              [-5/3, 5/3, -2/3]])


In [ ]:
# Собственная реализация SVD с использованием np.linalg.eig
def my_svd(A):
    ATA = A.T @ A
    eigenvalues, V = np.linalg.eig(ATA)
    idx = eigenvalues.argsort()[::-1] # возвращает индексы для сортировки
    eigenvalues = eigenvalues[idx]
    V = V[:, idx]
    singular_values = np.sqrt(eigenvalues)
    U = np.dot(A, V) / singular_values
    return U, singular_values, V.T


In [ ]:
# Сверим с готовой функцией
print(my_svd(A))
print('\n', np.linalg.svd(A))

(array([[-0.5       , -0.67082039,  0.2236068 ],
       [ 0.5       ,  0.2236068 ,  0.67082039],
       [-0.5       ,  0.2236068 ,  0.67082039],
       [ 0.5       , -0.67082039,  0.2236068 ]]), array([4., 2., 2.]), array([[-0.66666667,  0.66666667,  0.33333333],
       [ 0.        , -0.4472136 ,  0.89442719],
       [-0.74535599, -0.59628479, -0.2981424 ]]))

 SVDResult(U=array([[-0.5       ,  0.67082039, -0.2236068 , -0.5       ],
       [ 0.5       , -0.2236068 , -0.67082039, -0.5       ],
       [-0.5       , -0.2236068 , -0.67082039,  0.5       ],
       [ 0.5       ,  0.67082039, -0.2236068 ,  0.5       ]]), S=array([4., 2., 2.]), Vh=array([[-0.66666667,  0.66666667,  0.33333333],
       [-0.        ,  0.4472136 , -0.89442719],
       [ 0.74535599,  0.59628479,  0.2981424 ]]))


In [ ]:
# Попытка реализации с полного нуля
def delete(A, x, y):
    n = len(A)
    B = [[1 for j in range(n - 1)] for i in range(n - 1)]
    for i in range(n):
        if i == x:
            continue
        for j in range(n):
            if j == y:
                continue
            new_i = i if i < x else i - 1
            new_j = j if j < y else j - 1
            B[new_i][new_j] = A[i][j]
    return B

In [ ]:
B = A @ A.T
n = B.shape[0]
array = [[B[i][j] if j != i else np.poly1d([-1, B[i][i]]) for j in range(n)] for i in range(n)]
print(B)
def determinant(A):
    n = len(A)
    if n == 1:
        return A[0][0]
    elif n == 2:
        return np.polysub(np.polymul(A[0][0], A[1][1]), np.polymul(A[0][1], A[1][0]))
    else:
        det = np.poly1d([0])
        for i in range(n):
            M = delete(A, 0, i)
            det = np.polyadd(np.polymul((-1) ** i * A[0][i], determinant(M)), det)
        return det
char_poly = determinant(array)
char_poly = np.round(char_poly)
print(f'коэф. характ. мн-на: {char_poly}')

[[ 6. -4.  4. -2.]
 [-4.  6. -2.  4.]
 [ 4. -2.  6. -4.]
 [-2.  4. -4.  6.]]
коэф. характ. мн-на: [   1.  -24.  144. -256.    0.]


In [ ]:
eigenvalues = np.round(np.roots(char_poly))
print(eigenvalues)

B = np.array(B)
eigenvectors = []
for eigenvalue in eigenvalues:
    eigenvector = np.linalg.solve((B - eigenvalue * np.identity(n)), np.zeros(n))
    eigenvectors.append(eigenvector)

for i, eigenvector in enumerate(eigenvectors):
    print(f"λ = {eigenvalues[i]}:", eigenvector)
# собственные значения совпали с истинными, но вот вектора не посчитались, т.к. матрица вырожденная.

[16.  4.  4.  0.]
λ = 16.0: [-0. -0. -0. -0.]
λ = 4.0: [-0. -0.  0.  0.]
λ = 4.0: [-0. -0.  0.  0.]
λ = 0.0: [0. 0. 0. 0.]


In [ ]:
# еще попытка....
# Вырожденная матрицы
A = np.array([[1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]], dtype=np.float64)

# Ступенчатый вид
def gauss(matrix):
    n = len(matrix)
    for i in range(n):
        coeff = matrix[i][i]
        if coeff == 0: continue
        for h in range(n):
            matrix[i][h] /= coeff
        #print(A)
        for j in range(n):
            if i != j:
                temp = matrix[j][i]
                for p in range(n):
                    matrix[j][p] -= matrix[i][p] * temp
                #print(A)
    return matrix

print(gauss(A))

# Ядро матрицы
def ker(matrix):
    reduced_matrix = gauss(matrix)
    n = len(matrix)
    kernel = []
    for i in range(n):
        if np.allclose(reduced_matrix[i], np.zeros(n)):
            vector = np.zeros(n)
            vector[i] = 1
            for j in range(i):
                vector -= reduced_matrix[j] * matrix[j][i]
            kernel.append(vector)
    return kernel

kernel = ker(A)
print("Ядро вырожденной матрицы:")
for i in kernel:
    print(i)


[[ 1.  0. -1.]
 [-0.  1.  2.]
 [ 0.  0.  0.]]
Ядро вырожденной матрицы:
[ 1. -2. -4.]


## (8-10) Работа с датасетом

Будем использовать [MovieLens 100k](http://grouplens.org/datasets/movielens/) (*MovieLens Latest Datasets (small)*, ratings.csv)

Вам нужно:
- скачать датасет
- предобработать данные: составить список уникальных пользователей и фильмов
- составить матрицу **users - movies** (функция *create_utility_matrix*); значения матрицы - Nan, если нет информации об оценке пользователем фильма, или число, если оценка есть
- составить сингулярное разложение (используя пакетную функцию или собственную реализацию)
- выбрать одного пользователя (один фильм) и найти максимально похожих на него
- вывести рекомендации для пользователя


In [ ]:
#PATH = ### YOUR CODE HERE ###

data = pd.read_csv('/content/ratings.csv')
data['userId'] = data['userId'].astype('str')
data['movieId'] = data['movieId'].astype('str')
print(data)

# list of all users
users = data['userId']
# list of all movies
movies = data['movieId']

       userId movieId  rating   timestamp
0           1       1     4.0   964982703
1           1       3     4.0   964981247
2           1       6     4.0   964982224
3           1      47     5.0   964983815
4           1      50     5.0   964982931
...       ...     ...     ...         ...
100831    610  166534     4.0  1493848402
100832    610  168248     5.0  1493850091
100833    610  168250     5.0  1494273047
100834    610  168252     5.0  1493846352
100835    610  170875     3.0  1493846415

[100836 rows x 4 columns]


In [ ]:
def create_utility_matrix(data, formatizer = {'user':0, 'item': 1, 'value': 2}):
    """
        :param data:      Array-like, 2D, nx3
        :param formatizer:pass the formatizer
        :return:          utility matrix (n x m), n=users, m=items
    """

    itemField = formatizer['item']
    userField = formatizer['user']
    valueField = formatizer['value']

    userList = data.iloc[:,userField].tolist()
    itemList = data.iloc[:,itemField].tolist()
    valueList = data.iloc[:,valueField].tolist()

    users = list(set(data.iloc[:,userField]))
    items = list(set(data.iloc[:,itemField]))

    users_index = {users[i]: i for i in range(len(users))}

    pd_dict = {item: [np.nan for i in range(len(users))] for item in items}

    for i in range(0, len(data)):
        item = itemList[i]
        user = userList[i]
        value = valueList[i]
        pd_dict[item][users_index[user]] = value

    X = pd.DataFrame(pd_dict)
    X.index = users

    itemcols = list(X.columns)
    X = X.replace(np.nan, 0)
    items_index = {itemcols[i]: i for i in range(len(itemcols))}

    # users_index gives us a mapping of user_id to index of user
    # items_index provides the same for items

    return X, users_index, items_index

In [ ]:
matrix_useful, users_index, items_index = create_utility_matrix(data)
matrix_useful

,147326,93512,3580,1500,27124,5237,5346,88179,2306,7698,...,3510,1296,7492,69988,1748,26631,48560,7191,64983,1299
560,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
480,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
435,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Готовая
def do_svd(mat, k=0, option=False):
    U, Sigma, VT = np.linalg.svd(mat)
    U = pd.DataFrame(U[:,:k])
    VT = pd.DataFrame(VT[:k,:])
    if option:
        return Sigma
    else:
        return U, VT

In [ ]:
# Собственная реализация, но на больший данных она работает гораздо медленнее чем готовая svd
# svd может выполниться в пределах 2-3 минут, а вот из-за eig функция растягивается до 13 минут
def my_svd(A):
    ATA = A.T @ A
    eigenvalues, V = np.linalg.eig(ATA)
    idx = eigenvalues.argsort()[::-1] # возвращает индексы для сортировки
    eigenvalues = eigenvalues[idx]
    V = V[:, idx]
    singular_values = np.sqrt(eigenvalues)
    U = np.dot(A, V) / singular_values
    return U, singular_values, V.T


def my_do_svd(mat, k = 0, option = False):
    U, Sigma, VT = my_svd(mat)
    U = pd.DataFrame(U[:,:k])
    VT = pd.DataFrame(VT[:k,:])
    if option:
        return Sigma
    else:
        return U, VT

In [ ]:
#print(do_svd(matrix_useful, 10))
#print(my_do_svd(matrix_useful, 10)) тут убедились, что наша реализация сработала через eig

In [ ]:
def recommend_movie_item(liked_movie, VT, output_num=4):
    global rec
    rec = []
    for item in range(len(VT.columns)):
        if item != liked_movie:
            rec.append([item,np.dot(VT[item],VT[liked_movie])])
    final_rec = [i[0] for i in sorted(rec, key=lambda x: x[1],reverse=True)]
    return final_rec[:output_num]

In [ ]:
def recommend_movie_user(target_user, U, output_num=4): # по похожести пользователей
    global rec
    users_similarity = []
    for user in range(U.shape[0]):
        if user != target_user:
            users_similarity.append([user, np.dot(U.iloc[user], U.iloc[target_user])])
    sorted_users = [i[0] for i in sorted(users_similarity, key=lambda x: x[1],reverse=True)]
    all_movies = np.array(matrix_useful.iloc[sorted_users[0]])
    rec_movies = all_movies.nonzero()
    return rec_movies[0][:output_num]

In [ ]:
U, VT = my_do_svd(matrix_useful, 10)
print(f'Фильмы, похожие на 1 фильм {recommend_movie_item(1, VT)}')
print(f'Фильмы, рекомендованные для 1 пользователя {recommend_movie_user(1, U)}')

Фильмы, похожие на 1 фильм [1441, 3288, 9696, 5447]
Фильмы, рекомендованные для 1 пользователя [ 3 11 13 16]


In [ ]:
U, VT = do_svd(matrix_useful, 10)
print(f'Фильмы, похожие на 1 фильм {recommend_movie_item(1, VT)}')
print(f'Фильмы, рекомендованные для 1 пользователя {recommend_movie_user(1, U)}')

#тут сравним с готовой do_svd

Фильмы, похожие на 1 фильм [1441, 3288, 9696, 5447]
Фильмы, рекомендованные для 1 пользователя [ 3 11 13 16]
